## Project 1 (kmr8wyd)
#### Importing the Necessary Libraries

In [58]:
import os
import pandas as pd
import numpy

from sqlalchemy import create_engine

#### Loading CSV and Viewing Original File

In [59]:
data_dir = os.path.join(os.getcwd(), 'downloads')
data_file = os.path.join(data_dir, 'City_Trails.csv')

df = pd.read_csv(data_file, header=0, index_col=0)
df.head()

,NAME,TYPE,STATUS,EASEMENT,PROPERTY_OWNER
OBJECTID,,,,,
1,Greenbrier School Entry Path,Soft Surface,Existing,Complete,City School Board
10,Greenleaf - Walker School,Soft Surface,Existing,Complete,City Parks
13,250 Parallel,Shared Use Path,Existing,Complete,City
15,250 Parallel,Shared Use Path,Proposed,Complete,City
17,CHS McI Connector,Soft Surface,Proposed,Complete,City School Board


#### Modifying DataFrame

In [60]:
df_new = df.rename(columns={'PROPERTY_OWNER': 'PROPERTY OWNER'})
del df_new['EASEMENT']
df_new.insert(0, 'ID', range(1, 1 + len(df_new)))
# Get the DataFrame column names as a list
Dlist = ['ID','NAME', 'STATUS', 'PROPERTY OWNER','TYPE']

# Pass the new list to the DataFrame - like a key list in a dict 
dfnew = df_new[Dlist]
dfnew.head()

,ID,NAME,STATUS,PROPERTY OWNER,TYPE
OBJECTID,,,,,
1,1,Greenbrier School Entry Path,Existing,City School Board,Soft Surface
10,2,Greenleaf - Walker School,Existing,City Parks,Soft Surface
13,3,250 Parallel,Existing,City,Shared Use Path
15,4,250 Parallel,Proposed,City,Shared Use Path
17,5,CHS McI Connector,Proposed,City School Board,Soft Surface


#### Declare & Assign Connection Variables for the MySQL Server & Databases

In [61]:
host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

user_id = "root"
pwd = "AmericanahSummer21!"

dst_dbname = "trailinfo_db"

#### Defining a Function for Setting Data Into Databases

In [62]:
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [63]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

#### Setting Data into Database

In [64]:
try:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dfnew, 'dfnew', 'ID', "insert")
except:
    print("Error: Unable to set dataframe.")

#### Converting Data to JSON

In [68]:
try:
    dfnew.to_json('cvilletrails.json', orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)
except:
    print("Error: Unable to create JSON file.")

#### Returning Data Summary

In [71]:
num_records = dfnew.shape[0]
print ("There are ", num_records, "records in this data set.")
num_columns = len(dfnew.columns)
print ("There are ", num_columns, "columns in this data set.")

There are  500 records in this data set.
There are  5 columns in this data set.
